# Data loading 
Next cells are used to load the data we need.

In [ ]:
!git clone #clone your repo from git
!mv Recommender-Systems-Challenge/* ../content/

fatal: destination path 'Recommender-Systems-Challenge' already exists and is not an empty directory.
mv: cannot stat 'Recommender-Systems-Challenge/*': No such file or directory


In [ ]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 11 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1822:0,
                 from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorization_Cython_Epoch.c:598:
/usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it with " \
  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization_Cython_Epoch_32Matrix

In [ ]:
pip install implicit

In [ ]:
pip install optuna

In [ ]:
import os
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import sys
import matplotlib.pyplot as plt
import optuna


from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from MatrixFactorization.IALSRecommender_implicit import IALSRecommender_implicit

In [ ]:
dataset = pd.read_csv('/content/New Challenge Data/data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [ ]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

test_users = pd.read_csv('/content/New Challenge Data/data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [ ]:
ICM_df = pd.read_csv('/content/New Challenge Data/data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [ ]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [ ]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Base.Evaluation.Evaluator import EvaluatorHoldout

URM_train_1, URM_validation_1 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 1)

URM_train_2, URM_validation_2 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 2)

URM_train_3, URM_validation_3 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 3)

URM_train_4, URM_validation_4 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 4)

URM_train_5, URM_validation_5 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 5)

URM_train_vector = [URM_train_1,URM_train_2,URM_train_3,URM_train_4,URM_train_5]

evaluator_validation_1 = EvaluatorHoldout(URM_validation_1, cutoff_list=[10])
evaluator_validation_2 = EvaluatorHoldout(URM_validation_2, cutoff_list=[10])
evaluator_validation_3 = EvaluatorHoldout(URM_validation_3, cutoff_list=[10])
evaluator_validation_4 = EvaluatorHoldout(URM_validation_4, cutoff_list=[10])
evaluator_validation_5 = EvaluatorHoldout(URM_validation_5, cutoff_list=[10])

Evaluator_vector = [evaluator_validation_1, evaluator_validation_2, evaluator_validation_3, evaluator_validation_4, evaluator_validation_5]

In [ ]:
#{'topK': 635, 'weight': 0.28158935555171627, 'lr': 1.001891587185431e-05, 'epochs': 150, 'batch_size': 40}
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
 
class Objective(object):
    def __init__(self, URM_train_vector, ICM_all, Evaluator_vector):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train_vector = URM_train_vector
        self.ICM_all = ICM_all
        self.Evaluator_vector = Evaluator_vector
        self.MAP_vector = np.zeros(5)
 
        
 
    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        topK = trial.suggest_int('topK', 400,900)
        weight = trial.suggest_uniform('weight', 0.4, 0.6)
        lr = trial.suggest_uniform('lr', 1e-8, 1e-6)
        epochs= trial.suggest_int('epochs', 60,100)
        batch_size = trial.suggest_int('batch_size', 1,50)
        lambda_i = trial.suggest_uniform('lambda_i', 1e-5, 1e-3)
        lambda_j = trial.suggest_uniform('lambda_j', 1e-8, 1e-6)
 
        self.MAP_vector = np.zeros(5)
        
        i = 0
        for URM_train, evaluator_validation in zip(self.URM_train_vector,self.Evaluator_vector):
            URM_train_ICM_all = sps.vstack([URM_train*(1-weight), ICM_all.T*weight])
            recommender = SLIM_BPR_Cython(URM_train_ICM_all, recompile_cython=False, verbose=False)
            recommender.fit(epochs=epochs, sgd_mode= "adam", learning_rate=lr, topK = topK, batch_size = batch_size, lambda_i=lambda_i, lambda_j=lambda_j )
            result_dict, _ = evaluator_validation.evaluateRecommender(recommender)
            self.MAP_vector[i]=result_dict[10]["MAP"]
            i = i+1
        
        print('printing self map vector: ', self.MAP_vector)
        MAP = np.sum(self.MAP_vector)/5
        print('printing MAP: ', MAP)
 
 
 
        
        return MAP
    
# Execute an optimization by using an `Objective` instance.
study = optuna.create_study(direction='maximize')
study.optimize(Objective(URM_train_vector, ICM_all, Evaluator_vector), n_trials=100)
 
print(study.best_params)

[I 2021-01-12 23:19:53,133] A new study created in memory with name: no-name-4e508751-4125-4f9a-be8b-130508c44f7c


SLIM_BPR_Recommender: Epoch 1 of 93. Elapsed time 0.11 sec
SLIM_BPR_Recommender: Epoch 2 of 93. Elapsed time 0.24 sec
SLIM_BPR_Recommender: Epoch 3 of 93. Elapsed time 0.35 sec
SLIM_BPR_Recommender: Epoch 4 of 93. Elapsed time 0.48 sec
SLIM_BPR_Recommender: Epoch 5 of 93. Elapsed time 0.60 sec
SLIM_BPR_Recommender: Epoch 6 of 93. Elapsed time 0.73 sec
SLIM_BPR_Recommender: Epoch 7 of 93. Elapsed time 0.86 sec
SLIM_BPR_Recommender: Epoch 8 of 93. Elapsed time 0.98 sec
SLIM_BPR_Recommender: Epoch 9 of 93. Elapsed time 1.10 sec
SLIM_BPR_Recommender: Epoch 10 of 93. Elapsed time 1.22 sec
SLIM_BPR_Recommender: Epoch 11 of 93. Elapsed time 1.34 sec
SLIM_BPR_Recommender: Epoch 12 of 93. Elapsed time 1.46 sec
SLIM_BPR_Recommender: Epoch 13 of 93. Elapsed time 1.58 sec
SLIM_BPR_Recommender: Epoch 14 of 93. Elapsed time 1.70 sec
SLIM_BPR_Recommender: Epoch 15 of 93. Elapsed time 1.82 sec
SLIM_BPR_Recommender: Epoch 16 of 93. Elapsed time 1.95 sec
SLIM_BPR_Recommender: Epoch 17 of 93. Elapsed tim

[I 2021-01-12 23:27:15,285] Trial 0 finished with value: 0.05936474608720309 and parameters: {'topK': 429, 'weight': 0.5066803578773139, 'lr': 1.8571640677348022e-07, 'epochs': 93, 'batch_size': 7, 'lambda_i': 3.742468231956162e-05, 'lambda_j': 9.674076030677076e-07}. Best is trial 0 with value: 0.05936474608720309.


printing self map vector:  [0.06492538 0.05858233 0.06014283 0.05838598 0.05478721]
printing MAP:  0.05936474608720309
Deallocating Cython objects
SLIM_BPR_Recommender: Epoch 1 of 100. Elapsed time 0.12 sec
SLIM_BPR_Recommender: Epoch 2 of 100. Elapsed time 0.24 sec
SLIM_BPR_Recommender: Epoch 3 of 100. Elapsed time 0.36 sec
SLIM_BPR_Recommender: Epoch 4 of 100. Elapsed time 0.49 sec
SLIM_BPR_Recommender: Epoch 5 of 100. Elapsed time 0.62 sec
SLIM_BPR_Recommender: Epoch 6 of 100. Elapsed time 0.74 sec
SLIM_BPR_Recommender: Epoch 7 of 100. Elapsed time 0.86 sec
SLIM_BPR_Recommender: Epoch 8 of 100. Elapsed time 0.98 sec
SLIM_BPR_Recommender: Epoch 9 of 100. Elapsed time 1.10 sec
SLIM_BPR_Recommender: Epoch 10 of 100. Elapsed time 1.22 sec
SLIM_BPR_Recommender: Epoch 11 of 100. Elapsed time 1.34 sec
SLIM_BPR_Recommender: Epoch 12 of 100. Elapsed time 1.47 sec
SLIM_BPR_Recommender: Epoch 13 of 100. Elapsed time 1.59 sec
SLIM_BPR_Recommender: Epoch 14 of 100. Elapsed time 1.71 sec
SLIM_BPR

[I 2021-01-12 23:36:18,924] Trial 1 finished with value: 0.05971965091836401 and parameters: {'topK': 764, 'weight': 0.5563752910725698, 'lr': 1.1916648825674385e-07, 'epochs': 100, 'batch_size': 16, 'lambda_i': 0.0007356536511917711, 'lambda_j': 2.394084098169227e-07}. Best is trial 1 with value: 0.05971965091836401.


...

printing self map vector:  [0.06429525 0.05941952 0.06120238 0.05927721 0.05420796]
printing MAP:  0.0596804662335344
Deallocating Cython objects
SLIM_BPR_Recommender: Epoch 1 of 96. Elapsed time 0.16 sec
SLIM_BPR_Recommender: Epoch 2 of 96. Elapsed time 0.30 sec
SLIM_BPR_Recommender: Epoch 3 of 96. Elapsed time 0.45 sec
SLIM_BPR_Recommender: Epoch 4 of 96. Elapsed time 0.60 sec
SLIM_BPR_Recommender: Epoch 5 of 96. Elapsed time 0.75 sec
SLIM_BPR_Recommender: Epoch 6 of 96. Elapsed time 0.91 sec
SLIM_BPR_Recommender: Epoch 7 of 96. Elapsed time 1.05 sec
SLIM_BPR_Recommender: Epoch 8 of 96. Elapsed time 1.20 sec
SLIM_BPR_Recommender: Epoch 9 of 96. Elapsed time 1.34 sec
SLIM_BPR_Recommender: Epoch 10 of 96. Elapsed time 1.48 sec
SLIM_BPR_Recommender: Epoch 11 of 96. Elapsed time 1.61 sec
SLIM_BPR_Recommender: Epoch 12 of 96. Elapsed time 1.75 sec
SLIM_BPR_Recommender: Epoch 13 of 96. Elapsed time 1.90 sec
SLIM_BPR_Recommender: Epoch 14 of 96. Elapsed time 2.05 sec
SLIM_BPR_Recommender: E

[I 2021-01-13 10:55:26,004] Trial 75 finished with value: 0.0591022796717073 and parameters: {'topK': 741, 'weight': 0.5808959479582049, 'lr': 7.56355930044198e-07, 'epochs': 96, 'batch_size': 10, 'lambda_i': 0.0006195557554432999, 'lambda_j': 2.097173917270628e-07}. Best is trial 38 with value: 0.059742020434864304.


printing self map vector:  [0.06366419 0.05907062 0.05979387 0.05831293 0.05466978]
printing MAP:  0.0591022796717073
Deallocating Cython objects
SLIM_BPR_Recommender: Epoch 1 of 90. Elapsed time 0.14 sec
SLIM_BPR_Recommender: Epoch 2 of 90. Elapsed time 0.28 sec
SLIM_BPR_Recommender: Epoch 3 of 90. Elapsed time 0.42 sec
SLIM_BPR_Recommender: Epoch 4 of 90. Elapsed time 0.56 sec
SLIM_BPR_Recommender: Epoch 5 of 90. Elapsed time 0.70 sec
SLIM_BPR_Recommender: Epoch 6 of 90. Elapsed time 0.84 sec
SLIM_BPR_Recommender: Epoch 7 of 90. Elapsed time 0.97 sec
SLIM_BPR_Recommender: Epoch 8 of 90. Elapsed time 1.11 sec
SLIM_BPR_Recommender: Epoch 9 of 90. Elapsed time 1.25 sec
SLIM_BPR_Recommender: Epoch 10 of 90. Elapsed time 1.39 sec
SLIM_BPR_Recommender: Epoch 11 of 90. Elapsed time 1.52 sec
SLIM_BPR_Recommender: Epoch 12 of 90. Elapsed time 1.67 sec
SLIM_BPR_Recommender: Epoch 13 of 90. Elapsed time 1.81 sec
SLIM_BPR_Recommender: Epoch 14 of 90. Elapsed time 1.94 sec
SLIM_BPR_Recommender: E

[I 2021-01-13 11:04:40,566] Trial 76 finished with value: 0.059089643372198006 and parameters: {'topK': 713, 'weight': 0.591143115681488, 'lr': 8.950390863494438e-07, 'epochs': 90, 'batch_size': 15, 'lambda_i': 0.0005434370533209805, 'lambda_j': 1.722386907318727e-07}. Best is trial 38 with value: 0.059742020434864304.


printing self map vector:  [0.06335635 0.05856854 0.05979501 0.05958627 0.05414204]
printing MAP:  0.059089643372198006
Deallocating Cython objects
SLIM_BPR_Recommender: Epoch 1 of 94. Elapsed time 0.13 sec
SLIM_BPR_Recommender: Epoch 2 of 94. Elapsed time 0.28 sec
SLIM_BPR_Recommender: Epoch 3 of 94. Elapsed time 0.41 sec
SLIM_BPR_Recommender: Epoch 4 of 94. Elapsed time 0.55 sec
SLIM_BPR_Recommender: Epoch 5 of 94. Elapsed time 0.69 sec
SLIM_BPR_Recommender: Epoch 6 of 94. Elapsed time 0.83 sec
SLIM_BPR_Recommender: Epoch 7 of 94. Elapsed time 0.97 sec
SLIM_BPR_Recommender: Epoch 8 of 94. Elapsed time 1.11 sec
SLIM_BPR_Recommender: Epoch 9 of 94. Elapsed time 1.25 sec
SLIM_BPR_Recommender: Epoch 10 of 94. Elapsed time 1.38 sec
SLIM_BPR_Recommender: Epoch 11 of 94. Elapsed time 1.52 sec
SLIM_BPR_Recommender: Epoch 12 of 94. Elapsed time 1.66 sec
SLIM_BPR_Recommender: Epoch 13 of 94. Elapsed time 1.79 sec
SLIM_BPR_Recommender: Epoch 14 of 94. Elapsed time 1.92 sec
SLIM_BPR_Recommender: